# Required installations

Run once.

In [0]:
!pip install cvlib
%tensorflow_version 1.x

In [0]:
# When using for 1the first time make sure you
#         log in with the teams account when the confirmation URL is requested.
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

# Imports

In [0]:
import cv2
import glob
import cvlib as cv
from google.colab.patches import cv2_imshow

from keras import Model
import tensorflow as tf
#from tensorflow import keras
import keras #Need to be solved
from keras.layers import (Activation, Conv3D, Dense, Dropout, Flatten, 
                          MaxPooling3D, Lambda)
from keras.layers.advanced_activations import LeakyReLU
from keras.losses import categorical_crossentropy
from keras.layers.merge import concatenate
from keras.models import Sequential

from keras.optimizers import Adam
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model

import os
import numpy as np
import scipy
from scipy import ndimage
from random import shuffle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Helper Functions

## Loading Videos

This functions loads the videos and returns them in a list.

In [0]:
def read_videos(video_paths: list) -> list:
    videos = []
    for video_path in video_paths:
      videos.append(cv2.VideoCapture(video_path))
    cv2.destroyAllWindows()
    return videos

## Label Extraction

* based on the videos names we assign a label, 0 for deceptive and 1 for truthful.



In [0]:
def video_labeling(video_paths: list) -> list:
    video_label = []
    for video_path in video_paths:
        if 'lie' in video_path:
            video_label.append(0)
        else:
            video_label.append(1)
    return video_label

## Converting RGB videos to gray-scale

this function takes a list of videos and returns a list of gray-scale videos.

In [0]:
def rgb2gray(rgb_videos: list, resize_shape: tuple=(40, 60)) -> list:
    gray_videos = []
    for video in rgb_videos:
        # create a new video to act as the output gray video.
        
        # Read the first frame to find the original video attributes.
        ret, first_frame = video.read()
        cur_video_frames = []
        while(video.isOpened()):
            # Reading the next frame.
            ret, cur_frame = video.read()
            if not ret: 
                break 
            # Convert the RGB frame to gray-scale.
            cur_frame = cv2.cvtColor(cur_frame, cv2.COLOR_BGR2GRAY)
            # resizing the frame to (60 * 40)
            cur_frame = cv2.resize(cur_frame, resize_shape)
            # write the converted gray frame into the output video.
            cur_video_frames.append(cur_frame.tolist())
            # show the converted frame for testing
            #cv2_imshow(cur_frame)

            # to insure that there's more frames to be read.
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        # closing the opened video.
        video.release()
        # delete all opened frames.
        cv2.destroyAllWindows()
        # append the converted video to the list of videos to be returned.
        gray_videos.append(cur_video_frames)

    return gray_videos
        

## Crop a Layer dimension
Crops (or slices) a Tensor on a given dimension from start to end

example : to crop tensor x[:, :, 5:10] call slice(2, 5, 10) as you want to crop on the third dimension

In [0]:
def crop(dimension, start, end):
    # Crops (or slices) a Tensor on a given dimension from start to end
    # example : to crop tensor x[:, :, 5:10]
    # call slice(2, 5, 10) as you want to crop on the third dimension
    def func(x):
        if dimension == 0:
            return x[start: end]
        if dimension == 1:
            return x[:, start: end]
        if dimension == 2:
            return x[:, :, start: end]
        if dimension == 3:
            return x[:, :, :, start: end]
        if dimension == 4:
            return x[:, :, :, :, start: end]
    return Lambda(func)

# Data Loading

Defining the dataset path.


loading the videos names.

calling the read_videos function to read the videos into respective folders.

## Loading the Old Dataset Format

In [0]:
# Will be modified after dividing the videos into train and test
# Drives' path.
path = os.getcwd() + "/gdrive/My Drive/Team's Drive/Graduation Project/Project/Dataset/Clips/"

# Deceptive and truthful folder's pathes
deceptive_path = path + "Deceptive/"
truthful_path  = path + "Truthful/"

# Loading the videos names with their extension from the drive.
total_videos_batch = glob.glob(deceptive_path + '*[0-9].mp4') + glob.glob(truthful_path + '*[0-9].mp4')

# Shuffling the read videos to insure diversity.
shuffle(total_videos_batch)

# Getting the label column for the read videos.
video_label = video_labeling(total_videos_batch)

# Loading the actual vidios
input_videos = read_videos(total_videos_batch)

# converting the original RGB input videos to Gray-scale.
input_videos = rgb2gray(input_videos[:2])
#input_videos = rgb2gray(input_videos[:1])

In [0]:
input_videos = read_videos(total_videos_batch)
input_videos = rgb2gray(input_videos[:2])

print("videos", type(input_videos))
print("frames", type(input_videos[0]))
print("rows", type(input_videos[0][0]), len(input_videos[0][0]))
print("cols", type(input_videos[0][0][0]), len(input_videos[0][0][0]))
print(type(input_videos[0][0][0][0]))

input_videos = np.asarray(input_videos) # Invalid, different frames number
print(input_videos.shape)
print(type(input_videos[0]))

# Model Architecture

## Hard-wired Layer

* Take steps of 7 frames.
    
    * Copy volume of gray frames. *(7 feature maps)*
    * Apply gradient filter on the x-axis per frame. *(7 feature maps)*
    * Apply gradient filter on the y-axis per frame. *(7 feature maps)*
    * Extract optical flow channels for each 2 consecutive frames in both x and y. *(6 feature maps each)*
    * Concatenate the resulting volumes, 5 channels.
**Input dimensions (7@60x40)**

**Output dimensions (33@60x40)**, 33 feature maps.

In [0]:
def H1(cur_volume: np.ndarray(shape=(7, 60, 40))):
    volume = cur_volume
    prev_frame = None
    for frame in cur_volume:
        # For each frame, calculate the frame gradients
        """
        # Reshaping the frame to 4D tensorflow onject.
        tf_frame = tf.reshape(frame, shape = (1, frame.shape[1], frame.shape[0], 1))
        # Using the tensorflow built-in image_gradients function.
        g_x, g_y = tf.image.image_gradients(tf_frame)

        # Convert the tensorflow object to a numpy array to manipulate it easily.
        g_x = g_x.eval(session=tf.compat.v1.Session())
        # Reshape the numpy array to have the channels first.
        g_x = np.reshape(g_x, (1, 60, 40))
        # Append the current x_gradient to the output volume.
        volume = np.concatenate((volume, g_x), axis=0)

        # Convert the tensorflow object to a numpy array to manipulate it easily.
        g_y = g_y.eval(session=tf.compat.v1.Session())
        # Reshape the numpy array to have the channels first.
        g_y = np.reshape(g_y, (1, 60, 40))
        # Append the current y_gradient to the output volume.
        volume = np.concatenate((volume, g_y),  axis=0)
        """

        # Using the scipy library to get the image gradient in the x-axis.
        g_x = ndimage.sobel(frame, axis=0, mode='constant')
        #cv2_imshow(g_x)
        # Reshape the numpy array to have the channels first.
        g_x = np.reshape(g_x, (1, g_x.shape[0], g_x.shape[1]))
        # Append the current x_gradient to the output volume.
        volume = np.concatenate((volume, g_x), axis=0)

        # Using the scipy library to get the image gradient in the y-axis.
        g_y = ndimage.sobel(frame, axis=1, mode='constant')
        #cv2_imshow(g_y)
        # Reshape the numpy array to have the channels first.
        g_y = np.reshape(g_y, (1, g_y.shape[0], g_y.shape[1]))
        # Append the current y_gradient to the output volume.
        volume = np.concatenate((volume, g_y),  axis=0)

    # Extract the optical flow maps on the x-axis.
    prev_frame = None
    for frame in cur_volume:
        # For every 2 fames calculate the optical flow map between them.
        if prev_frame is not None:
            # Using the cv2 built-in function to get the optical flow map, 
            # given the previous frame, the current one.
            flow = cv2.calcOpticalFlowFarneback(prev_frame, frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            # Output is a numpy array with 2 channels, 
            # [0] is the optical flow map on the x axis 
            # Reshape the output optical map on the to have the channels first.
            flow_x = np.reshape(flow[:, :, 0], (1, 60, 40))
            # Append the calculated maps to the putput volume.
            volume = np.concatenate((volume, flow_x),  axis=0)
        prev_frame = frame

    prev_frame = None
    for frame in cur_volume:
        # For every 2 fames calculate the optical flow map between them.
        if prev_frame is not None:
            # Using the cv2 built-in function to get the optical flow map, 
            # given the previous frame, the current one.
            flow = cv2.calcOpticalFlowFarneback(prev_frame, frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            # Output is a numpy array with 2 channels, 
            # [1] is the optical flow map on the y-axis.
            # Reshape the output optical map on the to have the channels first.
            flow_y = np.reshape(flow[:, :, 1], (1, 60, 40))
            # Append the calculated maps to the putput volume.
            volume = np.concatenate((volume, flow_y),  axis=0)
        prev_frame = frame

    # print(volume.shape)
    return volume

In [0]:
#print(input_videos[0][0].shape)
#tmp = H1(np.asarray([input_videos[0][0]]))

## First Convolutional Layer

* On each channel apply a kernal **(7x7x3)** twice.
*(7x7 in spatial domain, 3 in the temporal)

**Input (33@60x40)**

**Output (2*23@54x34)**

In [0]:
def C2(input):
    out_set = None
    print("C2 volumes:")
    # kernel_size(depth, (kernel_shape))
    gray   = Conv3D(1, activation='tanh', kernel_size=(3, 7, 7), padding='valid', 
                 input_shape=(7, 60, 40, 1), data_format='channels_last')(crop(1, 0, 7)(input))
    print("gray", gray.shape)
    grad_x = Conv3D(1, activation='tanh', kernel_size=(3, 7, 7), padding='valid', 
                 input_shape=(7, 60, 40, 1), data_format='channels_last')(crop(1, 7, 14)(input))
    print("grad-x", grad_x.shape)
    grad_y = Conv3D(1, activation='tanh', kernel_size=(3, 7, 7), padding='valid', 
                 input_shape=(7, 60, 40, 1), data_format='channels_last')(crop(1, 14, 21)(input))
    print("grad-y", grad_y.shape)
    opt_x  = Conv3D(1, activation='tanh', kernel_size=(3, 7, 7), padding='valid', 
                 input_shape=(6, 60, 40, 1), data_format='channels_last')(crop(1, 21, 27)(input))
    print("opt-x", opt_x.shape)
    opt_y  = Conv3D(1, activation='tanh', kernel_size=(3, 7, 7), padding='valid', 
                 input_shape=(6, 60, 40, 1), data_format='channels_last')(crop(1, 27, 33)(input))
    print("opt-y", opt_y.shape)    
    # concatencation of the 5 channels is done on the second/dim-1 
    # which is the dimenion of frames, all are put in one set.
    out_set = concatenate([gray, grad_x, grad_y, opt_x, opt_y], axis = 1)
    print("out set:", out_set.shape)    
    return out_set

## Apply subsampling **(max-pooling 2x2)**

**Output (2*23@27x17)**

In [0]:
# to conserve the depth, to not be affected by the downsampling, we but it = 1.
def S3(input):
    out_set = MaxPooling3D(pool_size=(1, 2, 2))(input)
    return out_set

## Second Convolutional Layer

* On each channel apply a kernal **(7x6x3)** thrice.

**Output (6*13@21x12)**

In [0]:
def C4(input):
    out_set = None 
    print("C4 volumes:")
    # kernel_size(depth, (kernel_shape))
    gray   = Conv3D(1, activation='tanh', kernel_size=(3, 7, 6), padding='valid', 
                 input_shape=(5, 27, 17, 1), data_format='channels_last')(crop(1, 0, 5)(input))
    print("gray", gray.shape)
    grad_x = Conv3D(1, activation='tanh', kernel_size=(3, 7, 6), padding='valid', 
                 input_shape=(5, 27, 17, 1), data_format='channels_last')(crop(1, 5, 10)(input))
    print("grad-x", grad_x.shape)
    grad_y = Conv3D(1, activation='tanh', kernel_size=(3, 7, 6), padding='valid', 
                 input_shape=(5, 27, 17, 1), data_format='channels_last')(crop(1, 10, 15)(input))
    print("grad-y", grad_y.shape)
    opt_x  = Conv3D(1, activation='tanh', kernel_size=(3, 7, 6), padding='valid', 
                 input_shape=(4, 27, 17, 1), data_format='channels_last')(crop(1, 15, 19)(input))
    print("opt-x", opt_x.shape)
    opt_y  = Conv3D(1, activation='tanh', kernel_size=(3, 7, 6), padding='valid', 
                 input_shape=(4, 27, 17, 1), data_format='channels_last')(crop(1, 19, 23)(input))
    print("opt-y", opt_y.shape)    
    # concatencation of the 5 channels is done on the second/dim-1 
    # which is the dimenion of frames, all are put in one set.
    out_set = concatenate([gray, grad_x, grad_y, opt_x, opt_y], axis = 1)
    print("out set:", out_set.shape) 
    return out_set

## Apply subsampling **(max-pooling 3x3)**

**Output (6*13@7x4)**

In [0]:
def S5(input):
    output = MaxPooling3D(pool_size=(1, 3, 3))(input)
    return output

## Third Convolutional Layer
    
* On each channel apply a kernal **(7x4)**

**Output (78@1x1)**

In [0]:
def C6(input):
    out_set = None 
    print("C6 volumes:")
    # kernel_size(depth, (kernel_shape))
    gray   = Conv3D(1, activation='tanh', kernel_size=(1, 7, 4), padding='valid', 
                 input_shape=(3, 7, 4, 1), data_format='channels_last')(crop(1, 0, 3)(input))
    print("gray", gray.shape)
    grad_x = Conv3D(1, activation='tanh', kernel_size=(1, 7, 4), padding='valid', 
                 input_shape=(3, 7, 4, 1), data_format='channels_last')(crop(1, 3, 6)(input))
    print("grad-x", grad_x.shape)
    grad_y = Conv3D(1, activation='tanh', kernel_size=(1, 7, 4), padding='valid', 
                 input_shape=(3, 7, 4, 1), data_format='channels_last')(crop(1, 6, 9)(input))
    print("grad-y", grad_y.shape)
    opt_x  = Conv3D(1, activation='tanh', kernel_size=(1, 7, 4), padding='valid', 
                 input_shape=(2, 7, 4, 1), data_format='channels_last')(crop(1, 9, 11)(input))
    print("opt-x", opt_x.shape)
    opt_y  = Conv3D(1, activation='tanh', kernel_size=(1, 7, 4), padding='valid', 
                 input_shape=(2, 7, 4, 1), data_format='channels_last')(crop(1, 11, 13)(input))
    print("opt-y", opt_y.shape)    
    # concatencation of the 5 channels is done on the second/dim-1 
    # which is the dimenion of frames, all are put in one set.
    out_set = concatenate([gray, grad_x, grad_y, opt_x, opt_y], axis = 1)
    print("out set:", out_set.shape) 
    return out_set

## FC-Layer of **(128@1x1)**
* Connect to output layer.

In [0]:
def FC(input:list):
    # input: a list of sets representing the last output from layer C6.
    output = Flatten()(input)
    output = Dense(units=128, activation='tanh')(output)
    return output

# Variables

* Input:
    * input_videos : list of vedios, dimensions (m, n, frame_rows, frame_cols).
    
    * data : numpy array, dimensions (m, n, 7, frame_rows, frame_cols).
    * m : #Vedios, n : #Frames in a vedio, (frame_rows, frame_cols) : Frame dimensions (40x60).

In [0]:
# input_videos = np.asarray(input_videos)
step = 6
inside_step = 2

# drunkly verified
num_of_volumes = ((input_videos[0].shape[0] - (step * inside_step))//step) + 1
train_data_shape = (input_videos.shape[0], num_of_volumes)

# N: "number videos" * "number volumes"
N = input_videos.shape[0] * num_of_volumes
x_train = np.empty((input_videos.shape[0], num_of_volumes, 33, 60, 40), dtype=object)
y_train = np.empty((input_videos.shape[0], num_of_volumes, 2), dtype=object)

# The loop
for vid_ind in range(len(input_videos)):
    for frame_ind, vol_num in zip(range(step, len(input_videos[vid_ind]) - step, step), range(num_of_volumes)):
        first_frame = frame_ind - 6
        last_frame  = frame_ind + 6 + 1 # adding one for slicing

        # get the new volume from the hardwired level.
        #new_volume  = input_videos[vid_ind, first_frame:last_frame:inside_step, :, :]
        new_volume = H1(input_videos[vid_ind, first_frame:last_frame:inside_step, :, :])
        
        # Add the new volume to input of the first 3D-CNN layer
        x_train[vid_ind, vol_num, :, :, :] = new_volume

        # Create the label for this volume.
        # video_label carries the video label, 1 for truthful and 0 for deceptive.
        # our y_train has 2 cells, [0] deceptive and [1] for truthful
        cur_y = np.zeros((2), int)
        cur_y[video_label[vid_ind]] = 1
        #print(cur_y.shape)
        y_train[vid_ind, vol_num, :] = cur_y

# Reshape the data to (N, dimesions)
x_train = x_train.reshape((N, 33, 60, 40, 1))
y_train = y_train.reshape((N, 2))

print(x_train.shape, y_train.shape)
# print(y_train[0])

# Training

In [0]:
print(x_train.shape)
print(y_train.shape)